# DISH enrichment tables

## Import modules

In [1]:
# Import modules
import pandas as pd
import numpy as np
from datetime import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Load files

In [120]:
# Especify the path to the file
path_to_n1_acdm = 'C:/Users/alvaro.mendoza/Desktop/N1 ACDM Export/20220825113059.csv'
path_to_n1_rfciq = 'C:/Users/alvaro.mendoza/Desktop/N1 RF CIQ_Table/20220825121623.csv'

In [121]:
def open_file(my_file_path):
    '''
    Function:   
        + Opens a csv file and converts it into a pandas dataframe with all fields read as string (except NaN)
    Input:
        + my_file_path  => Path of the file
    Output:
        + my_file_as_df => Pandas dataframe
    '''

    # Variable control
    assert isinstance(my_file_path, str), 'The path to the file must be as string'
    assert len(my_file_path) > 0, 'Path is empty'

    # Code
    with open(my_file_path) as my_file: # Opening this way it is not necessary to close the file to release resources
        my_file_as_df = pd.read_csv(my_file, dtype=str)
    
    return my_file_as_df

In [ ]:
def export_to_xlsx(path, df):
    with pd.ExcelWriter(path) as writer:
        df.to_excel(writer, sheet_name='py_export', index=False)

In [136]:
df_n1_acdm = open_file(path_to_n1_acdm)
df_n1_rfciq = open_file(path_to_n1_rfciq)

#n1_acdm_col = ['MCC', 'MNC', 'NR Tracking Area Code', 'NR Cell ID',	'Cell Site Common Name','Market (AOI)',
#                'City',	'State', 'Cell Name']
#n1_rfciq_col = ['Site ID', 'Sector ID', 'TX ID', 'CIQ MCC', 'CIQ MNC', 'NR CGI', 'NR Cell ID', 'NR Cell Name']

n1_acdm_col = ['NR Cell ID', 'Cell Site Common Name', 'Cell Name'] # 20220825113059
n1_rfciq_col = ['NR Cell ID','Site ID', 'TX ID'] # 20220825121623

df_n1_acdm = df_n1_acdm[n1_acdm_col]
df_n1_rfciq = df_n1_rfciq[n1_rfciq_col]

df_n1_acdm = df_n1_acdm.sort_values(by=['Cell Site Common Name', 'Cell Name'], ignore_index=True)
df_n1_rfciq = df_n1_rfciq.sort_values(by=['Site ID', 'TX ID'], ignore_index=True)

df_n1_acdm = df_n1_acdm.dropna(subset=['NR Cell ID'])
df_n1_rfciq = df_n1_rfciq.dropna(subset=['NR Cell ID'])

df_n1_acdm = df_n1_acdm.drop_duplicates(subset=['NR Cell ID'], keep='first')
df_n1_rfciq = df_n1_rfciq.drop_duplicates(subset=['NR Cell ID'], keep='first')

df_n1_acdm['NR Cell ID'] = df_n1_acdm['NR Cell ID'].astype(str)
df_n1_rfciq['NR Cell ID'] = df_n1_rfciq['NR Cell ID'].astype(str)

In [137]:
df_n1_acdm.head()

,NR Cell ID,Cell Site Common Name,Cell Name
0,04BEDD003,ALALB00003A,ALALB00003A_n29_E_DL_1
1,04BEDD004,ALALB00003A,ALALB00003A_n29_E_DL_2
2,04BEDD005,ALALB00003A,ALALB00003A_n29_E_DL_3
3,04BEDD009,ALALB00003A,ALALB00003A_n66_AWS-4_DL_1
4,04BEDD00A,ALALB00003A,ALALB00003A_n66_AWS-4_DL_2


In [138]:
df_n1_rfciq.head()

,NR Cell ID,Site ID,TX ID
0,1273872492,ALALB00001A,ALALB00001A_n29_E_DL_1
1,1273872493,ALALB00001A,ALALB00001A_n29_E_DL_2
2,1273872494,ALALB00001A,ALALB00001A_n29_E_DL_3
3,1273872498,ALALB00001A,ALALB00001A_n66_AWS-4_DL_1
4,1273872499,ALALB00001A,ALALB00001A_n66_AWS-4_DL_2


In [139]:
df_n1_acdm['NR Cell ID len'] = df_n1_acdm['NR Cell ID'].apply(len)
df_n1_acdm = df_n1_acdm[df_n1_acdm['NR Cell ID len'] == 9]
df_n1_acdm['NR Cell ID'] = df_n1_acdm['NR Cell ID'].str.lower()
df_n1_acdm['KEY'] = df_n1_acdm['Cell Name'] + '_' + df_n1_acdm['NR Cell ID']
df_n1_acdm = df_n1_acdm.drop_duplicates(subset=['KEY'], keep='first')

df_n1_acdm['NR CGI hex'] = '0x133304' + df_n1_acdm['NR Cell ID']
df_n1_acdm['NR CGI hex'] = df_n1_acdm['NR CGI hex'].astype(str)
df_n1_acdm['NR CGI dec'] = df_n1_acdm['NR CGI hex'].apply(int, base=16)
df_n1_acdm

,NR Cell ID,Cell Site Common Name,Cell Name,NR Cell ID len,KEY,NR CGI hex,NR CGI dec
0,04bedd003,ALALB00003A,ALALB00003A_n29_E_DL_1,9,ALALB00003A_n29_E_DL_1_04bedd003,0x13330404bedd003,86465870560088067
1,04bedd004,ALALB00003A,ALALB00003A_n29_E_DL_2,9,ALALB00003A_n29_E_DL_2_04bedd004,0x13330404bedd004,86465870560088068
2,04bedd005,ALALB00003A,ALALB00003A_n29_E_DL_3,9,ALALB00003A_n29_E_DL_3_04bedd005,0x13330404bedd005,86465870560088069
3,04bedd009,ALALB00003A,ALALB00003A_n66_AWS-4_DL_1,9,ALALB00003A_n66_AWS-4_DL_1_04bedd009,0x13330404bedd009,86465870560088073
4,04bedd00a,ALALB00003A,ALALB00003A_n66_AWS-4_DL_2,9,ALALB00003A_n66_AWS-4_DL_2_04bedd00a,0x13330404bedd00a,86465870560088074
...,...,...,...,...,...,...,...
250224,069b8711e,SYSYR00553A,SYSYR00553A_n70_AWS-4_UL15_2,9,SYSYR00553A_n70_AWS-4_UL15_2_069b8711e,0x133304069b8711e,86465871059906846
250225,069b8711f,SYSYR00553A,SYSYR00553A_n70_AWS-4_UL15_3,9,SYSYR00553A_n70_AWS-4_UL15_3_069b8711f,0x133304069b8711f,86465871059906847
250226,069b87117,SYSYR00553A,SYSYR00553A_n71_F-G_1,9,SYSYR00553A_n71_F-G_1_069b87117,0x133304069b87117,86465871059906839
250227,069b87118,SYSYR00553A,SYSYR00553A_n71_F-G_2,9,SYSYR00553A_n71_F-G_2_069b87118,0x133304069b87118,86465871059906840


In [140]:
df_n1_rfciq['NR Cell ID hex'] = pd.to_numeric(df_n1_rfciq['NR Cell ID']).apply(hex)
df_n1_rfciq['NR Cell ID hex'] = df_n1_rfciq['NR Cell ID hex'].astype(str)
df_n1_rfciq['NR Cell ID hex'] = df_n1_rfciq['NR Cell ID hex'].str.replace(r'x', '')
df_n1_rfciq['NR Cell ID hex len'] = df_n1_rfciq['NR Cell ID hex'].apply(len)
df_n1_rfciq = df_n1_rfciq[df_n1_rfciq['NR Cell ID hex len'] == 9]
df_n1_rfciq['KEY'] = df_n1_rfciq['TX ID'] + '_' + df_n1_rfciq['NR Cell ID hex']
df_n1_rfciq = df_n1_rfciq.drop_duplicates(subset=['KEY'], keep='first')

df_n1_rfciq['NR CGI hex'] = '0x133304' + df_n1_rfciq['NR Cell ID']
df_n1_rfciq['NR CGI hex'] = df_n1_rfciq['NR CGI hex'].astype(str)
df_n1_rfciq['NR CGI dec'] = df_n1_rfciq['NR CGI hex'].apply(int, base=16)
df_n1_rfciq

,NR Cell ID,Site ID,TX ID,NR Cell ID hex,NR Cell ID hex len,KEY,NR CGI hex,NR CGI dec
0,1273872492,ALALB00001A,ALALB00001A_n29_E_DL_1,04bedc06c,9,ALALB00001A_n29_E_DL_1_04bedc06c,0x1333041273872492,1383453987827033234
1,1273872493,ALALB00001A,ALALB00001A_n29_E_DL_2,04bedc06d,9,ALALB00001A_n29_E_DL_2_04bedc06d,0x1333041273872493,1383453987827033235
2,1273872494,ALALB00001A,ALALB00001A_n29_E_DL_3,04bedc06e,9,ALALB00001A_n29_E_DL_3_04bedc06e,0x1333041273872494,1383453987827033236
3,1273872498,ALALB00001A,ALALB00001A_n66_AWS-4_DL_1,04bedc072,9,ALALB00001A_n66_AWS-4_DL_1_04bedc072,0x1333041273872498,1383453987827033240
4,1273872499,ALALB00001A,ALALB00001A_n66_AWS-4_DL_2,04bedc073,9,ALALB00001A_n66_AWS-4_DL_2_04bedc073,0x1333041273872499,1383453987827033241
...,...,...,...,...,...,...,...,...
625097,1773613342,SYSYR00554A,SYSYR00554A_n70_AWS-4_UL15_2,069b7311e,9,SYSYR00554A_n70_AWS-4_UL15_2_069b7311e,0x1333041773613342,1383454009299383106
625098,1773613343,SYSYR00554A,SYSYR00554A_n70_AWS-4_UL15_3,069b7311f,9,SYSYR00554A_n70_AWS-4_UL15_3_069b7311f,0x1333041773613343,1383454009299383107
625099,1773613335,SYSYR00554A,SYSYR00554A_n71_A_1,069b73117,9,SYSYR00554A_n71_A_1_069b73117,0x1333041773613335,1383454009299383093
625100,1773613336,SYSYR00554A,SYSYR00554A_n71_A_2,069b73118,9,SYSYR00554A_n71_A_2_069b73118,0x1333041773613336,1383454009299383094


In [141]:
df_n1_acdm.head()

,NR Cell ID,Cell Site Common Name,Cell Name,NR Cell ID len,KEY,NR CGI hex,NR CGI dec
0,04bedd003,ALALB00003A,ALALB00003A_n29_E_DL_1,9,ALALB00003A_n29_E_DL_1_04bedd003,0x13330404bedd003,86465870560088067
1,04bedd004,ALALB00003A,ALALB00003A_n29_E_DL_2,9,ALALB00003A_n29_E_DL_2_04bedd004,0x13330404bedd004,86465870560088068
2,04bedd005,ALALB00003A,ALALB00003A_n29_E_DL_3,9,ALALB00003A_n29_E_DL_3_04bedd005,0x13330404bedd005,86465870560088069
3,04bedd009,ALALB00003A,ALALB00003A_n66_AWS-4_DL_1,9,ALALB00003A_n66_AWS-4_DL_1_04bedd009,0x13330404bedd009,86465870560088073
4,04bedd00a,ALALB00003A,ALALB00003A_n66_AWS-4_DL_2,9,ALALB00003A_n66_AWS-4_DL_2_04bedd00a,0x13330404bedd00a,86465870560088074


In [142]:
df_n1_rfciq.head()

,NR Cell ID,Site ID,TX ID,NR Cell ID hex,NR Cell ID hex len,KEY,NR CGI hex,NR CGI dec
0,1273872492,ALALB00001A,ALALB00001A_n29_E_DL_1,04bedc06c,9,ALALB00001A_n29_E_DL_1_04bedc06c,0x1333041273872492,1383453987827033234
1,1273872493,ALALB00001A,ALALB00001A_n29_E_DL_2,04bedc06d,9,ALALB00001A_n29_E_DL_2_04bedc06d,0x1333041273872493,1383453987827033235
2,1273872494,ALALB00001A,ALALB00001A_n29_E_DL_3,04bedc06e,9,ALALB00001A_n29_E_DL_3_04bedc06e,0x1333041273872494,1383453987827033236
3,1273872498,ALALB00001A,ALALB00001A_n66_AWS-4_DL_1,04bedc072,9,ALALB00001A_n66_AWS-4_DL_1_04bedc072,0x1333041273872498,1383453987827033240
4,1273872499,ALALB00001A,ALALB00001A_n66_AWS-4_DL_2,04bedc073,9,ALALB00001A_n66_AWS-4_DL_2_04bedc073,0x1333041273872499,1383453987827033241


In [144]:
# Set name and path for the exported file
today_date = datetime.today().strftime('%Y-%m-%d')
today_date = today_date.replace('-', '')

file_name_acdm = today_date + ' - 20220825113059' + '.xlsx'
export_path_acdm = 'C:/Users/alvaro.mendoza/Desktop/N1 ACDM Export/'
file_exp_path_acdm = export_path_acdm + file_name_acdm

file_name_rfciq = today_date + ' - 20220825121623' + '.xlsx'
export_path_rfciq = 'C:/Users/alvaro.mendoza/Desktop/N1 RF CIQ_Table/'
file_exp_path_rfciq = export_path_rfciq + file_name_rfciq


In [145]:
export_to_xlsx(file_exp_path_acdm, df_n1_acdm)

In [146]:
export_to_xlsx(file_exp_path_rfciq, df_n1_rfciq)

In [6]:
# Set name and path for the exported file
today_date = datetime.today().strftime('%Y-%m-%d')
today_date = today_date.replace('-', '')
file_name = today_date + ' - 20220825113059' + '.xlsx'
export_path = 'C:/Users/alvaro.mendoza/Desktop/N1 ACDM Export/'
file_exp_path = export_path + file_name

with pd.ExcelWriter(file_exp_path) as writer:
    df_n1_acdm.to_excel(writer, sheet_name='export', index=False)

In [8]:
# Set name and path for the exported file
today_date = datetime.today().strftime('%Y-%m-%d')
today_date = today_date.replace('-', '')
file_name = today_date + ' - 20220825121623' + '.xlsx'
export_path = 'C:/Users/alvaro.mendoza/Desktop/N1 RF CIQ_Table/'
file_exp_path = export_path + file_name

with pd.ExcelWriter(file_exp_path) as writer:
    df_n1_rfciq.to_excel(writer, sheet_name='export', index=False)

In [9]:
df_n1_acdm.head()

,ADD/CHANGE/DELETE,Technology,MCC,MNC,NR Tracking Area Code,NR Cell ID,gNodeB Identity Bit Length,NR PCI,Cell Site Common Name,Carrier Cell ID Cascade ID,...,Vertical Beam Width,Sector Radius,Unit of Radius,Antenna Tilt,DAS,Repeater,Antenna Manufacturer,Antenna Model #,Small Cell,Cell Name
0,A,NR,313,340,031102,04BEDD003,24,147,ALALB00003A,NaN,...,15,8046,Meters,NaN,NaN,NaN,NaN,MX08FRO665-21,No,ALALB00003A_n29_E_DL_1
1,A,NR,313,340,031102,04BEDD004,24,148,ALALB00003A,NaN,...,15,8046,Meters,NaN,NaN,NaN,NaN,MX08FRO665-21,No,ALALB00003A_n29_E_DL_2
2,A,NR,313,340,031102,04BEDD005,24,149,ALALB00003A,NaN,...,15,8046,Meters,NaN,NaN,NaN,NaN,MX08FRO665-21,No,ALALB00003A_n29_E_DL_3
3,A,NR,313,340,031102,04BEDD009,24,147,ALALB00003A,NaN,...,5,8046,Meters,NaN,NaN,NaN,NaN,MX08FRO665-21,No,ALALB00003A_n66_AWS-4_DL_1
4,A,NR,313,340,031102,04BEDD00A,24,148,ALALB00003A,NaN,...,5,8046,Meters,NaN,NaN,NaN,NaN,MX08FRO665-21,No,ALALB00003A_n66_AWS-4_DL_2


In [10]:
df_n1_rfciq.head()

,Site ID,Sector ID,TX ID,CU UP ID,CU-CP ID,DU ID,DU Name,Local Cell ID,CIQ MCC,CIQ MNC,NR CGI,NR Cell ID,NR Cell Name,TAC,CIQ gNodeB ID,gNodeB Name,nSSAI
0,CHCHI00001A,n29_E_DL_1,CHCHI00001A_n29_E_DL_1,100,0,NaN,NaN,NaN,313,340,NaN,NaN,CHCHI00001A_1_n29_E,NaN,NaN,NaN,33554431
1,CHCHI00001A,n29_E_DL_2,CHCHI00001A_n29_E_DL_2,100,0,NaN,NaN,NaN,313,340,NaN,NaN,CHCHI00001A_2_n29_E,NaN,NaN,NaN,33554431
2,CHCHI00001A,n29_E_DL_3,CHCHI00001A_n29_E_DL_3,100,0,NaN,NaN,NaN,313,340,NaN,NaN,CHCHI00001A_3_n29_E,NaN,NaN,NaN,33554431
3,CHCHI00001A,n66_AWS-4_DL_1,CHCHI00001A_n66_AWS-4_DL_1,100,0,NaN,NaN,NaN,313,340,NaN,NaN,CHCHI00001A_1_n66_AWS4,NaN,NaN,NaN,33554431
4,CHCHI00001A,n66_AWS-4_DL_2,CHCHI00001A_n66_AWS-4_DL_2,100,0,NaN,NaN,NaN,313,340,NaN,NaN,CHCHI00001A_2_n66_AWS4,NaN,NaN,NaN,33554431


In [11]:
df_n1_acdm = df_n1_acdm[n1_acdm_col]
df_n1_acdm

,MCC,MNC,NR Tracking Area Code,NR Cell ID,Cell Site Common Name,Market (AOI),City,State,Cell Name
0,313,340,031102,04BEDD003,ALALB00003A,ALB,Clifton Park,NY,ALALB00003A_n29_E_DL_1
1,313,340,031102,04BEDD004,ALALB00003A,ALB,Clifton Park,NY,ALALB00003A_n29_E_DL_2
2,313,340,031102,04BEDD005,ALALB00003A,ALB,Clifton Park,NY,ALALB00003A_n29_E_DL_3
3,313,340,031102,04BEDD009,ALALB00003A,ALB,Clifton Park,NY,ALALB00003A_n66_AWS-4_DL_1
4,313,340,031102,04BEDD00A,ALALB00003A,ALB,Clifton Park,NY,ALALB00003A_n66_AWS-4_DL_2
...,...,...,...,...,...,...,...,...,...
250224,313,340,043303,069B8711E,SYSYR00553A,SYR,Rochester,NY,SYSYR00553A_n70_AWS-4_UL15_2
250225,313,340,043303,069B8711F,SYSYR00553A,SYR,Rochester,NY,SYSYR00553A_n70_AWS-4_UL15_3
250226,313,340,043303,069B87117,SYSYR00553A,SYR,Rochester,NY,SYSYR00553A_n71_F-G_1
250227,313,340,043303,069B87118,SYSYR00553A,SYR,Rochester,NY,SYSYR00553A_n71_F-G_2


In [9]:
df_n1_rfciq = df_n1_rfciq[n1_rfciq_col]
df_n1_rfciq

,Site ID,Sector ID,TX ID,CIQ MCC,CIQ MNC,NR CGI,NR Cell ID,NR Cell Name
0,CHCHI00001A,n29_E_DL_1,CHCHI00001A_n29_E_DL_1,313,340,NaN,NaN,CHCHI00001A_1_n29_E
1,CHCHI00001A,n29_E_DL_2,CHCHI00001A_n29_E_DL_2,313,340,NaN,NaN,CHCHI00001A_2_n29_E
2,CHCHI00001A,n29_E_DL_3,CHCHI00001A_n29_E_DL_3,313,340,NaN,NaN,CHCHI00001A_3_n29_E
3,CHCHI00001A,n66_AWS-4_DL_1,CHCHI00001A_n66_AWS-4_DL_1,313,340,NaN,NaN,CHCHI00001A_1_n66_AWS4
4,CHCHI00001A,n66_AWS-4_DL_2,CHCHI00001A_n66_AWS-4_DL_2,313,340,NaN,NaN,CHCHI00001A_2_n66_AWS4
...,...,...,...,...,...,...,...,...
625292,CHCHI00913C,n70_AWS-4_UL15_2,CHCHI00913C_n70_AWS-4_UL15_2,313,340,NaN,NaN,CHCHI00913B_2_n70_AWS4_UL15
625293,CHCHI00913C,n70_AWS-4_UL15_3,CHCHI00913C_n70_AWS-4_UL15_3,313,340,NaN,NaN,CHCHI00913B_3_n70_AWS4_UL15
625294,CHCHI00913C,n71_G_1,CHCHI00913C_n71_G_1,313,340,NaN,NaN,CHCHI00913B_1_n71_G
625295,CHCHI00913C,n71_G_2,CHCHI00913C_n71_G_2,313,340,NaN,NaN,CHCHI00913B_2_n71_G


In [10]:
df_n1_acdm = df_n1_acdm.dropna(subset=['NR Cell ID'])
df_n1_rfciq = df_n1_rfciq.dropna(subset=['NR Cell ID'])

In [11]:
len(df_n1_acdm), len(df_n1_rfciq)

(206975, 303136)

In [12]:
df_n1_rfciq

,Site ID,Sector ID,TX ID,CIQ MCC,CIQ MNC,NR CGI,NR Cell ID,NR Cell Name
17328,DADAL00001A,n29_E_DL_1,DADAL00001A_n29_E_DL_1,313,340,8.64659E+16,2256900099,DADAL00001A_1_n29_E
17329,DADAL00001A,n29_E_DL_2,DADAL00001A_n29_E_DL_2,313,340,8.64659E+16,2256900100,DADAL00001A_2_n29_E
17330,DADAL00001A,n29_E_DL_3,DADAL00001A_n29_E_DL_3,313,340,8.64659E+16,2256900101,DADAL00001A_3_n29_E
17331,DADAL00001A,n66_AWS-4_DL_1,DADAL00001A_n66_AWS-4_DL_1,313,340,8.64659E+16,2256900105,DADAL00001A_1_n66_AWS4
17332,DADAL00001A,n66_AWS-4_DL_2,DADAL00001A_n66_AWS-4_DL_2,313,340,8.64659E+16,2256900106,DADAL00001A_2_n66_AWS4
...,...,...,...,...,...,...,...,...
625277,NYNYC01927C,n70_AWS-4_UL15_2,NYNYC01927C_n70_AWS-4_UL15_2,313,340,8.64659E+16,1437945974,NYNYC01927C_2_n70_AWS4_UL15
625278,NYNYC01927C,n70_AWS-4_UL15_3,NYNYC01927C_n70_AWS-4_UL15_3,313,340,8.64659E+16,1437945975,NYNYC01927C_3_n70_AWS4_UL15
625279,NYNYC01927C,n71_D-E-F-G_1,NYNYC01927C_n71_D-E-F-G_1,313,340,8.64659E+16,1437945967,NYNYC01927C_1_n71_D-E-F-G
625280,NYNYC01927C,n71_D-E-F-G_2,NYNYC01927C_n71_D-E-F-G_2,313,340,8.64659E+16,1437945968,NYNYC01927C_2_n71_D-E-F-G


In [13]:
df_n1_rfciq['NR Cell ID hex'] = pd.to_numeric(df_n1_rfciq['NR Cell ID']).apply(hex)
df_n1_rfciq

C:\Users\alvaro.mendoza\AppData\Local\Temp\ipykernel_25932\2881020013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n1_rfciq['NR Cell ID hex'] = pd.to_numeric(df_n1_rfciq['NR Cell ID']).apply(hex)


,Site ID,Sector ID,TX ID,CIQ MCC,CIQ MNC,NR CGI,NR Cell ID,NR Cell Name,NR Cell ID hex
17328,DADAL00001A,n29_E_DL_1,DADAL00001A_n29_E_DL_1,313,340,8.64659E+16,2256900099,DADAL00001A_1_n29_E,0x86859003
17329,DADAL00001A,n29_E_DL_2,DADAL00001A_n29_E_DL_2,313,340,8.64659E+16,2256900100,DADAL00001A_2_n29_E,0x86859004
17330,DADAL00001A,n29_E_DL_3,DADAL00001A_n29_E_DL_3,313,340,8.64659E+16,2256900101,DADAL00001A_3_n29_E,0x86859005
17331,DADAL00001A,n66_AWS-4_DL_1,DADAL00001A_n66_AWS-4_DL_1,313,340,8.64659E+16,2256900105,DADAL00001A_1_n66_AWS4,0x86859009
17332,DADAL00001A,n66_AWS-4_DL_2,DADAL00001A_n66_AWS-4_DL_2,313,340,8.64659E+16,2256900106,DADAL00001A_2_n66_AWS4,0x8685900a
...,...,...,...,...,...,...,...,...,...
625277,NYNYC01927C,n70_AWS-4_UL15_2,NYNYC01927C_n70_AWS-4_UL15_2,313,340,8.64659E+16,1437945974,NYNYC01927C_2_n70_AWS4_UL15,0x55b55076
625278,NYNYC01927C,n70_AWS-4_UL15_3,NYNYC01927C_n70_AWS-4_UL15_3,313,340,8.64659E+16,1437945975,NYNYC01927C_3_n70_AWS4_UL15,0x55b55077
625279,NYNYC01927C,n71_D-E-F-G_1,NYNYC01927C_n71_D-E-F-G_1,313,340,8.64659E+16,1437945967,NYNYC01927C_1_n71_D-E-F-G,0x55b5506f
625280,NYNYC01927C,n71_D-E-F-G_2,NYNYC01927C_n71_D-E-F-G_2,313,340,8.64659E+16,1437945968,NYNYC01927C_2_n71_D-E-F-G,0x55b55070


In [6]:
hex(86465870240587700)

'0x133304038e29fb4'

In [75]:
int(0x133304069B87119)

86465871059906841

In [ ]:
# TX ID and Cell Name will be the KEY
1 drop all samples without NR_CellID
3059 => 206975 samples
1623 => 303136 samples
2 drop all unused columns
3 dec2hex the NR_CellID if needed
4 columns homogenization (cell_name and site_name)
5 create the nRCGI as per .pdf
5 create new key => cell_name+NR_CellID_hex

In [ ]:
What I saw
1) the files have different cell_name (some of them are the same, but just few)
2) there are NR_CellID missing => we should drop them as they are part of the nRCGI
3) we can generate the nRCGI just with the NR_CellID (mnc and mcc we already know)
4) then we can "combine" the two files and remove duplicates and do the vlookup


In [ ]:
20220825121623
Steps
delete all columns except Site ID, TX ID (actully Cell Name) and NR Cell ID
Dish Network is MCC = 313 + MNC = 340 allways


20220825113059
Steps
delete all columns except Site ID, Cell Name) and NR Cell ID
Dish Network is MCC = 313 + MNC = 340 allways

re-arrange both of them



In [3]:
# Create DataFrame for 2G Nokia data & clean it up
df_nok_2g = pd.read_excel(xls, sheet_name='NOK_2G', dtype=str) #Read specific sheet | dtype=str forces to read as string to avoid lose '0s'
df_nok_2g['New_CGI'] = df_nok_2g['para_MCC'] + '-' + df_nok_2g['para_MNC'] + '-' + df_nok_2g['para_LAC'] + '-' + df_nok_2g['para_CI/SA']    # Create col New_CGI
df_nok_2g['DE-PARA'] = 'de NE=' + df_nok_2g['NE'] + ' - LAC=' + df_nok_2g['LAC'] + ' para NE=' + df_nok_2g['para_NE'] + ' - LAC=' + df_nok_2g['para_LAC']                                   # Create col DE-PARA
df_nok_2g = df_nok_2g.sort_values(by=['MSS', 'Municipio', 'NE', 'CELL_NAME'], ignore_index=True)                            # Sort DataFrame

In [4]:
# Create DataFrame for 3G Nokia data & clean it up
df_nok_3g = pd.read_excel(xls, sheet_name='NOK_3G', dtype=str) #Read specific sheet | dtype=str forces to read as string to avoid lose '0s'
df_nok_3g['New_CGI'] = df_nok_3g['para_MCC'] + '-' + df_nok_3g['para_MNC'] + '-' + df_nok_3g['para_LAC'] + '-' + df_nok_3g['para_CI/SA']    # Create col New_CGI
df_nok_3g['DE-PARA'] = 'de NE=' + df_nok_3g['NE'] + ' - LAC=' + df_nok_3g['LAC'] + ' para NE=' + df_nok_3g['para_NE'] + ' - LAC=' + df_nok_3g['para_LAC']                                   # Create col DE-PARA
df_nok_3g = df_nok_3g.sort_values(by=['MSS', 'Municipio', 'NE', 'CELL_NAME'], ignore_index=True)                            # Sort DataFrame

## Print 2G Nokia's (change & fallback) commands

In [5]:
# Get 2G Nokia's change commands
NOK_ERI_Toolkit.rehome_cell_2g_nok(my_df=df_nok_2g, my_cmd='change')


! AMARGOSA | Change de NE=BSCBA48 - LAC=2075 para NE=BSCBA43 - LAC=675
!--------------------------------------------------------------------------------------
ZEPC:NAME=BAAMR011,NO=73000:MCC=724,MNC=05,LAC=675,CI=50189;
ZEPC:NAME=BAAMR012,NO=73001:MCC=724,MNC=05,LAC=675,CI=50190;
ZEPC:NAME=BAAMR013,NO=73002:MCC=724,MNC=05,LAC=675,CI=50191;
ZEPC:NAME=BAAMR014,NO=73003:MCC=724,MNC=05,LAC=675,CI=50195;
ZEPC:NAME=BAAMR015,NO=73004:MCC=724,MNC=05,LAC=675,CI=50196;
ZEPC:NAME=BAAMR016,NO=73005:MCC=724,MNC=05,LAC=675,CI=50192;
ZEPB:NAME=BAAMR011:BSCNAME=BSCBA43;
ZEPB:NAME=BAAMR012:BSCNAME=BSCBA43;
ZEPB:NAME=BAAMR013:BSCNAME=BSCBA43;
ZEPB:NAME=BAAMR014:BSCNAME=BSCBA43;
ZEPB:NAME=BAAMR015:BSCNAME=BSCBA43;
ZEPB:NAME=BAAMR016:BSCNAME=BSCBA43;
ZEPR:NAME=BAAMR011:RZ=1553,CDR=75;
ZEPR:NAME=BAAMR012:RZ=1553,CDR=75;
ZEPR:NAME=BAAMR013:RZ=1553,CDR=75;
ZEPR:NAME=BAAMR014:RZ=1553,CDR=75;
ZEPR:NAME=BAAMR015:RZ=1553,CDR=75;
ZEPR:NAME=BAAMR016:RZ=1553,CDR=75;
ZEPS:NAME=BAAMR011:U;
ZEPS:NAME=BAAMR012:U;
ZEPS

True

In [6]:
# Get 2G Nokia's fallback commands
NOK_ERI_Toolkit.rehome_cell_2g_nok(my_df=df_nok_2g, my_cmd='fallback')


! AMARGOSA | Fallback de NE=BSCBA48 - LAC=2075 para NE=BSCBA43 - LAC=675
!--------------------------------------------------------------------------------------
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferente de MSS destino - FB N/A - CleanUp em outra CRQ se necessario
MSS origem diferen

True

## Print 3G Nokia's (change & fallback) commands

In [7]:
# Get 3G Nokia's change commands
NOK_ERI_Toolkit.rehome_cell_3g_nok(my_df=df_nok_3g, my_cmd='change')


! ARACAJU | Change de NE=RNCSE02 - LAC=30379 para NE=RNCSE03 - LAC=30679
!--------------------------------------------------------------------------------------
ZEPO:TYPE=SA,NAME=USEAJU061;
ZEPO:TYPE=SA,NAME=USEAJU062;
ZEPO:TYPE=SA,NAME=USEAJU063;
ZEPO:TYPE=SA,NAME=USEAJU065;
ZEPO:TYPE=SA,NAME=USEAJU066;
ZEPO:TYPE=SA,NAME=USEAJU067;
ZEPO:TYPE=SA,NAME=USEAJU0803;
ZEPO:TYPE=SA,NAME=USEAJU081;
ZEPO:TYPE=SA,NAME=USEAJU082;
ZEPO:TYPE=SA,NAME=USEAJU085;
ZEPO:TYPE=SA,NAME=USEAJU0853;
ZEPO:TYPE=SA,NAME=USEAJU086;
ZEPO:TYPE=SA,NAME=USEAJU251;
ZEPO:TYPE=SA,NAME=USEAJU252;
ZEPO:TYPE=SA,NAME=USEAJU253;
ZEPO:TYPE=SA,NAME=USEAJU255;
ZEPO:TYPE=SA,NAME=USEAJU256;
ZEPO:TYPE=SA,NAME=USEAJU257;
ZEPO:TYPE=SA,NAME=USEAJU411;
ZEPO:TYPE=SA,NAME=USEAJU412;
ZEPO:TYPE=SA,NAME=USEAJU413;
ZEPO:TYPE=SA,NAME=USEAJU415;
ZEPO:TYPE=SA,NAME=USEAJU416;
ZEPO:TYPE=SA,NAME=USEAJU417;
ZEPO:TYPE=SA,NAME=USEAJU431;
ZEPO:TYPE=SA,NAME=USEAJU432;
ZEPO:TYPE=SA,NAME=USEAJU433;
ZEPO:TYPE=SA,NAME=USEAJU435;
ZEPO:TYPE=SA,NAME=USEAJU

True

In [8]:
# Get 3G Nokia's fallback commands
NOK_ERI_Toolkit.rehome_cell_3g_nok(my_df=df_nok_3g, my_cmd='fallback')


! ARACAJU | Fallback de NE=RNCSE02 - LAC=30379 para NE=RNCSE03 - LAC=30679
!--------------------------------------------------------------------------------------
ZEPO:TYPE=SA,NAME=USEAJU061;
ZEPO:TYPE=SA,NAME=USEAJU062;
ZEPO:TYPE=SA,NAME=USEAJU063;
ZEPO:TYPE=SA,NAME=USEAJU065;
ZEPO:TYPE=SA,NAME=USEAJU066;
ZEPO:TYPE=SA,NAME=USEAJU067;
ZEPO:TYPE=SA,NAME=USEAJU0803;
ZEPO:TYPE=SA,NAME=USEAJU081;
ZEPO:TYPE=SA,NAME=USEAJU082;
ZEPO:TYPE=SA,NAME=USEAJU085;
ZEPO:TYPE=SA,NAME=USEAJU0853;
ZEPO:TYPE=SA,NAME=USEAJU086;
ZEPO:TYPE=SA,NAME=USEAJU251;
ZEPO:TYPE=SA,NAME=USEAJU252;
ZEPO:TYPE=SA,NAME=USEAJU253;
ZEPO:TYPE=SA,NAME=USEAJU255;
ZEPO:TYPE=SA,NAME=USEAJU256;
ZEPO:TYPE=SA,NAME=USEAJU257;
ZEPO:TYPE=SA,NAME=USEAJU411;
ZEPO:TYPE=SA,NAME=USEAJU412;
ZEPO:TYPE=SA,NAME=USEAJU413;
ZEPO:TYPE=SA,NAME=USEAJU415;
ZEPO:TYPE=SA,NAME=USEAJU416;
ZEPO:TYPE=SA,NAME=USEAJU417;
ZEPO:TYPE=SA,NAME=USEAJU431;
ZEPO:TYPE=SA,NAME=USEAJU432;
ZEPO:TYPE=SA,NAME=USEAJU433;
ZEPO:TYPE=SA,NAME=USEAJU435;
ZEPO:TYPE=SA,NAME=USEA

True